In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
%cd drive/MyDrive/CSCE670_ISR/project/Animendations
!ls

/content/drive/MyDrive/CSCE670_ISR/project/Animendations
'Animendations Project Proposal.pdf'   LICENSE
 app.py				       preprocessed_data
 data				       Preprocessing.ipynb
 Final_Preprocessing.ipynb	       README.md
 Item-item.ipynb


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import ast

animes_data_path = "./preprocessed_data/new_preprocessed_animes.csv"
profiles_data_path = "./preprocessed_data/preprocessed_profiles.csv"
reviews_data_path = "./preprocessed_data/new_preprocessed_reviews.csv"

In [5]:
animes_data = pd.read_csv(animes_data_path)
profiles_data = pd.read_csv(profiles_data_path)
reviews_data = pd.read_csv(reviews_data_path, engine='python', sep=',', error_bad_lines=False)

<ipython-input-5-39739bd06a7d>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  reviews_data = pd.read_csv(reviews_data_path, engine='python', sep=',', error_bad_lines=False)


In [6]:
animes = animes_data.copy()
profiles = profiles_data.copy()
reviews = reviews_data.copy()

In [7]:
from scipy.sparse import coo_matrix
import numpy as np

np.random.seed(0)
unique_AnimeID = reviews['anime_uid'].unique()
unique_users = reviews['profile'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_users:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_AnimeID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
user_list = reviews['profile'].values
movie_list = reviews['anime_uid'].values
for j in range(len(reviews)):
    user_list[j] = user_old2new_id_dict[user_list[j]]
    movie_list[j] = movie_old2new_id_dict[movie_list[j]]
reviews['profile'] = user_list
reviews['anime_uid'] = movie_list

# generate train_df with 70% samples and test_df with 30% samples, and there should have no overlap between them.
train_index = np.random.random(len(reviews)) <= 0.7
train_df = reviews[train_index]
test_df = reviews[~train_index]

# generate train_mat and test_mat
num_user = len(reviews['profile'].unique())
num_items = len(reviews['anime_uid'].unique())

train_mat = coo_matrix((train_df['score'].values, (train_df['profile'].values, train_df['anime_uid'].values)), shape=(num_user, num_items)).astype(float).toarray()
test_mat = coo_matrix((test_df['score'].values, (test_df['profile'].values, test_df['anime_uid'].values)), shape=(num_user, num_items)).astype(float).toarray()

     

In [14]:
print(train_mat[2][:20])
print(test_mat[2][:20])

[0. 0. 7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
print(test_mat)

[[ 0.  8.  0. ...  0.  0.  0.]
 [ 0. 10.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
itemitem_cosine_similarities = cosine_similarity(train_mat.transpose())
print(itemitem_cosine_similarities)

[[1.         0.03892981 0.         ... 0.         0.         0.        ]
 [0.03892981 1.         0.01087724 ... 0.         0.         0.        ]
 [0.         0.01087724 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
itemitem = np.zeros((train_mat.shape[0], 2000))
valval = np.zeros((train_mat.shape[0], 2000))

for i in range(0, itemitem.shape[0]):
  for j in range(0, itemitem.shape[1]):
    itemitem[i][j] = -1

twenty_neighbours = np.argsort(itemitem_cosine_similarities, axis = 0)[::-1][1:101]

mean = np.average(train_mat)
b_u = np.average(train_mat, axis = 1)
b_i = np.average(train_mat, axis = 0)

for user in range(train_mat.shape[0]):
  remaining_items = np.where(train_mat[user] == 0)[0]
  remaining_items_rating_dict = dict()
  for item in remaining_items:
    b_ui = b_u[user] + b_i[item] - mean
    num = 0
    den = 0
    for n in twenty_neighbours[:, item]:
      num = num + (itemitem_cosine_similarities[item][n] * (train_mat[user][n] - b_ui))
      den = den + itemitem_cosine_similarities[item][n]
    remaining_items_rating_dict[item] = (num/den) + b_ui
  remaining_items_rating_dict = dict(sorted(remaining_items_rating_dict.items(), key = lambda k: k[1], reverse = True))

  i = 0
  for item in remaining_items_rating_dict:
    itemitem[user][i] = item
    valval[user][i] = remaining_items_rating_dict[item]
    i = i + 1
    if i == 2000:
      break
    
#print("Movie recommendation matrix created using item-item collaborative filtering") 
#print(itemitem)

<ipython-input-20-9f5d04fe6840>:24: RuntimeWarning: invalid value encountered in double_scalars
  remaining_items_rating_dict[item] = (num/den) + b_ui


In [21]:
print(itemitem.astype(int))

[[ 846  197  818 ... 2050 2057 2028]
 [3394   49   40 ... 1971 1974 1978]
 [ 846  639  474 ... 1992 1972 1977]
 ...
 [   4   13   17 ... 1956 1970 1980]
 [ 658  576  508 ... 1959 1974 1980]
 [ 107    3   13 ... 1969 1976 1980]]


In [22]:
print(itemitem.shape)

(10692, 2000)


In [23]:
print(valval)

[[ 1.98068699e+00  1.72069779e+00  1.54681626e+00 ...  4.86670575e-01
   2.37377042e-01  2.13594468e-01]
 [ 5.25480703e-01  4.26140532e-01  3.28421064e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 8.15466825e-01  3.14477465e-01  2.99629633e-01 ... -8.67361738e-19
  -1.30104261e-18 -2.16840434e-18]
 ...
 [ 8.38414139e-02             nan             nan ... -4.33680869e-19
  -4.33680869e-19 -4.33680869e-19]
 [ 1.66533454e-16  1.52655666e-16  8.32667268e-17 ... -4.33680869e-19
  -4.33680869e-19 -4.33680869e-19]
 [ 1.66533454e-16  5.55111512e-17             nan ... -8.67361738e-19
  -8.67361738e-19 -8.67361738e-19]]


In [24]:
print(remaining_items_rating_dict)

{107: 1.6653345369377348e-16, 3: 5.551115123125783e-17, 13: nan, 17: nan, 105: 4.163336342344337e-17, 23: 2.7755575615628914e-17, 44: 1.3877787807814457e-17, 46: 1.3877787807814457e-17, 98: 6.938893903907228e-18, 35: 3.469446951953614e-18, 9: 1.734723475976807e-18, 11: 1.734723475976807e-18, 24: 1.734723475976807e-18, 31: 1.734723475976807e-18, 113: 1.734723475976807e-18, 19: 1.3010426069826053e-18, 12: 8.673617379884035e-19, 27: 8.673617379884035e-19, 30: 8.673617379884035e-19, 34: 8.673617379884035e-19, 36: 6.505213034913027e-19, 106: 6.505213034913027e-19, 0: 4.336808689942018e-19, 14: 4.336808689942018e-19, 20: 4.336808689942018e-19, 33: 4.336808689942018e-19, 15: 2.710505431213761e-19, 7: 0.0, 18: 0.0, 32: 0.0, 37: -1.0842021724855044e-19, 10: -2.168404344971009e-19, 41: -2.168404344971009e-19, 42: -2.168404344971009e-19, 38: -3.2526065174565133e-19, 16: -4.336808689942018e-19, 21: -4.336808689942018e-19, 40: -4.336808689942018e-19, 28: -6.505213034913027e-19, 22: -1.3010426069826

In [25]:
predicted = np.zeros((train_mat.shape[0], train_mat.shape[1]))
for x in range(0, itemitem.shape[0]):
  for y in range(0, itemitem.shape[1]):
    if itemitem[x][y]!= 0 and valval[x][y] != np.NaN and valval[x][y] > 0.01:
      predicted[x][int(itemitem[x][y])] = valval[x][y]
print(predicted)

[[0.         0.3827894  0.07852322 ... 0.         0.         0.        ]
 [0.         0.28422408 0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# **RMSE**

In [26]:
import math
cnt = 0
sum = 0
for x in range(0, test_mat.shape[0]):
  for y in range(0, test_mat.shape[1]):
    if test_mat[x][y] != 0 and test_mat[x][y] != np.nan:
      a1 = float(test_mat[x][y])
      a2 = predicted[x][y]
      sum = sum + (a1-a2)*(a1-a2)
      #print(sum)
      if a1 == np.nan or a2 == np.nan or sum == np.nan:
        print(x, y)
      cnt = cnt + 1
    if cnt == 10:
      break
print(math.sqrt(sum/cnt))


7.614402097663336
